In [1]:
# import sys
# sys.path.insert(0, '../../src')

import quik_ai as qa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets

from importlib.metadata import version 
print('Quik-AI Version: %s' % version('quik_ai'))

Quik-AI Version: 1.2.0


In [2]:
iris = datasets.load_iris()

# Convert all data to DataFrame
sepal_values = iris.data[:, :2]
petal_values = iris.data[:, 2:4]

# Create new DataFrame
data = pd.DataFrame({
    "sepal": list(sepal_values),
    "petal": list(petal_values),
    "target": iris.target
})

data = data.sample(frac=1.0, random_state=1337).reset_index(drop=True)

data.head()

,sepal,petal,target
0,"[5.7, 2.9]","[4.2, 1.3]",1
1,"[5.0, 3.2]","[1.2, 0.2]",0
2,"[7.2, 3.2]","[6.0, 1.8]",2
3,"[4.7, 3.2]","[1.6, 0.2]",0
4,"[7.4, 2.8]","[6.1, 1.9]",2


In [3]:
training_data, testing_data, _ = qa.train_val_test_split(data, p=[0.8, 0.2, 0.0])

# create the driver, this will generate the tensorflow dataset and feed data
driver = qa.Driver(training_data, testing_data, testing_data, batch_size=5)

# what does our response look like?
head = qa.heads.Logistic(event_size=len(data.target.unique()))

# all the predictors for the model
predictors = [
    qa.predictors.Numerical('sepal'),
    qa.predictors.Numerical('petal')
]

# what is our model architecture? This will store
# our model, driver, and any other information we use in training
# testing, saving, and loading needed for this problem
model = qa.models.ResNet('target', head, predictors, driver)

# train the model, we can provide a hyperparameter tuner and other
# training parameters, by default we will use a custom BOHB tuner
model.train(tuner_params={'max_epochs' : 10})

Trial 30 Complete [00h 00m 05s]
val_sparse_categorical_crossentropy: 0.3695557713508606

Best val_sparse_categorical_crossentropy So Far: 0.04478077217936516
Total elapsed time: 00h 01m 44s
INFO:tensorflow:Oracle triggered exit
Round 1 best score: 0.0325


In [4]:
# evaluate the model on test data
model.evaluate()

6/6 [==============================] - 0s 6ms/step - loss: 0.0325 - sparse_categorical_crossentropy: 0.0325 - sparse_categorical_accuracy: 1.0000


{'loss': 0.032523415982723236,
 'sparse_categorical_crossentropy': 0.032523415982723236,
 'sparse_categorical_accuracy': 1.0}

In [5]:
# save the model to disk and reload the model
# then test on the test data once again
model.save()
model = qa.HyperModel.load()

# evaluate again, this should be the same score as before
model.evaluate()

INFO:tensorflow:Assets written to: ./model\tf_model\assets


INFO:tensorflow:Assets written to: ./model\tf_model\assets


6/6 [==============================] - 0s 6ms/step - loss: 0.0325 - sparse_categorical_crossentropy: 0.0325 - sparse_categorical_accuracy: 1.0000


{'loss': 0.032523415982723236,
 'sparse_categorical_crossentropy': 0.032523415982723236,
 'sparse_categorical_accuracy': 1.0}

In [6]:
model.instance.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sepal (InputLayer)             [(None, 2)]          0           []                               
                                                                                                  
 petal (InputLayer)             [(None, 2)]          0           []                               
                                                                                                  
 tf.identity_4 (TFOpLambda)     (None, 2)            0           ['sepal[0][0]']                  
                                                                                                  
 tf.identity_6 (TFOpLambda)     (None, 2)            0           ['petal[0][0]']                  
                                                                                            

In [7]:
results = pd.DataFrame({
    'prediction' : model.predict(testing_data, verbose=0).argmax(axis=1),
    'target' : testing_data.target,
})

results

,prediction,target
0,1,1
1,2,2
2,0,0
3,0,0
4,1,1
5,1,1
6,1,1
7,1,1
8,2,2
9,1,1
